# Parrot Diffusion Studies 🦜🎨

This notebook is designed to study parameters, settings and prompts against the [Stable Diffusion model](https://github.com/CompVis/stable-diffusion).

This notebook maintained by [Stephen Young](https://twitter.com/KyrickYoung) or SteveTheNinja#0616

Visit the Parrot Zone [parrotzone.art](http://www.parrotzone.art)

## Changelog
### 2.0
- paperspace port!

In [ ]:
#@title Setup
!pip install git+https://github.com/kyrick/parrot-tools.git --no-deps
!pip install transformers scipy ftfy diffusers

In [8]:
# @title Imports
from pathlib import Path
import os

from parrot_tools.utils import prepare_prompts_for_study, prepare_hybrid_prompts_for_study
from parrot_tools.generate import run_prompts, BatchSettings, set_scheduler

import torch
from torch import autocast
from diffusers import StableDiffusionPipeline

# THIS IS A DIRTY HACK TO SILENCE THE PROGRESS BAR
# THE TQDM PROGRESS BAR BOTTLENECKS CELL OUTPUT AND SLOWS THE NOTEBOOK
from tqdm.auto import tqdm
from functools import partialmethod
tqdm.__init__ = partialmethod(tqdm.__init__, disable=True)

# setup paths
NOTEBOOK_PATH = Path("/notebooks")
IMAGES_OUT = NOTEBOOK_PATH / "images_out"

os.makedirs(IMAGES_OUT, exist_ok=True)

# Log in to HuggingFace for access to models

In order to access the models from CompVis for Stable Diffusion, you must follow three steps:

1. You must acknowledge and agree to their user requirements and license for their models. you can do so by reading the instructions found on this page: https://huggingface.co/CompVis/stable-diffusion-v1-4

2. You must login to Huggingface, and then create and retrieve an access token (found here: https://huggingface.co/settings/tokens)

3. Finally, replace the segment of the cell below `<your_huggingface_token>` with your own token, and run the cell. 

If you follow these steps, you will be able to access the model for free!

In [10]:
# Add auth token here

token = "<your_huggingface_token>"

In [14]:
# Initialize the Model

model_name = "CompVis/stable-diffusion-v1-4"
pipe = StableDiffusionPipeline.from_pretrained(model_name, 
                                               revision="fp16", 
                                               torch_dtype=torch.float16, 
                                               use_auth_token=token).to("cuda")
    

In [ ]:
# Each count will add 1 additional style or modifier to each prompt. A positive number will flip the notebook into HYBRID MODE! :D
# Turn on hybridize_ALL_THE_THINGS to combine both styles and modifiers into the same prompts
hybrid_count = 0
hybridize_ALL_THE_THINGS = False

base_prompts = """
a portrait of a character in a scenic environment
a building in a stunning landscape
"""

# Supported formats:
# Gediminas Pranckevicius - first, last
# Aivazovsky, Ivan        - last, first (supports tabs if pasted from google sheets!)
# de Kooning, Willem      - 2+ words in the last name
# Death Burger            - a nickname of two or more words
# RHADS                   - a single nickname
styles_to_study = """
Pranckevicius, Gediminas
Aivazovsky	Ivan
RHADS
"""

modifiers_to_study = """
graphic novel
vaporwave
"""

# This text will be added to the end of every prompt
append_to_all_prompts = ""


if hybrid_count > 0:
    prompts_to_run = prepare_hybrid_prompts_for_study(base_prompts, styles_to_study, modifiers_to_study, append_to_all_prompts, hybrid_count=hybrid_count, hybridize_everything=hybridize_ALL_THE_THINGS)
else:
    prompts_to_run = prepare_prompts_for_study(base_prompts, styles_to_study, modifiers_to_study, append_to_all_prompts)

# print out the prompts to sanity check
for p in prompts_to_run:
    print(p.prompt, "-", p.base_filename)

In [ ]:
#@title Do The Run
#@markdown ## Run Settings
#@markdown set seed to -1 for random seed
batch_name = "StableStudy" #@param {type: "string"}
#@markdown batch_size is images per prompt
batch_size = 6 #@param {type: "integer"}
manual_seed = -1 #@param {type:"integer"}
steps = 50  #@param {type:"integer"}
cfg_scale = 7.0  #@param {type:"number"}
scheduler = "k_lms" #@param ['k_lms', 'ddim', 'pndm']

#@markdown set to 0 to turn off. Number of retries per NSFW result (will try with different seed)
NSFW_retries = 1 #@param {type:"integer"}

#@markdown ##Image Settings
image_ext = "jpeg" #@param ["jpeg", "png"]
width = 512  #@param {type: "integer"}
height = 512  #@param {type: "integer"}

#@markdown turn on/off dispalay of each image
display_single_images = True #@param {type:"boolean"}

#@markdown ##Grid Settings
make_batch_grid = False #@param {type:"boolean"}
grid_cols = -1 #@param {type:"integer"}
max_images_per_grid = 6 #@param {type:"integer"}
grid_padding = 5 #@param {type:"integer"}
#@markdown color name (`black`, `white`, `pink`, etc.) or hex code in format `#00004c'`
grid_bg_color = "black" #@param {type: "string"}

batch_settings = BatchSettings(
    batch_size=batch_size,
    batch_name=batch_name,
    base_path=IMAGES_OUT,
    steps=steps,
    scheduler=scheduler,
    cfg_scale=cfg_scale,
    seed=manual_seed,
    NSFW_retry=NSFW_retries,
    display_individual_images=display_single_images,
    image_ext=image_ext,
    image_w=width,
    image_h=height,
    make_grid=make_batch_grid,
    grid_cols=grid_cols,
    grid_max_images=max_images_per_grid,
    grid_padding=grid_padding,
    grid_bg_color=grid_bg_color
)

# from diffusers import LMSDiscreteScheduler, DDIMScheduler, PNDMScheduler
# lms = LMSDiscreteScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear")
# pipe.scheduler = lms
set_scheduler(pipe, batch_settings.scheduler)
run_prompts(pipe, prompts_to_run, batch_settings)
